In [1]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import importlib
importlib.reload(spot)
importlib.reload(roi_prop)
importlib.reload(intensity)

<module 'easi_fish.intensity' from 'D:\\SWAP\\code\\easifish-proc\\data_processing\\..\\easi_fish\\intensity.py'>

In [2]:
### Specify directories to data files
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
output_dir = os.path.join(input_dir, "testfull-aug5-v2")

# fixed image (directory - n5 format)
img_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 
# spot dir 
spot_dir = os.path.join(input_dir, "spots_pooled") # pool spots together; warpped and fixed
# bad roi
bad_roi = os.path.join(output_dir, "bad_roi_list.npy")
### end input

# output
out_roi = os.path.join(output_dir, "roi_all.csv") 
out_goodroi = os.path.join(output_dir, "roi.csv") 
out_spots = os.path.join(output_dir, "spots.csv")
out_intensity = os.path.join(output_dir, "intensity_c0_r2.csv")

In [3]:
### Load segmentation mask
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

number of ROIs in total: 13758
number of bad ROIs: 891


In [4]:
### Extract ROI properties
print(lb.shape)
# lb = lb[500:600,500:600,500:600]
# lb.shape # z, x,y 

(1019, 915, 916)


In [5]:
%%time
df = roi_prop.roi_prop_v2(lb)
df

CPU times: total: 18.5 s
Wall time: 18.6 s


,roi,z,y,x,area
0,1,363.489055,17.368291,73.106487,102.469416
1,2,125.553784,51.560521,396.296073,1692.922728
2,3,111.095382,104.351716,254.707325,1133.917848
3,4,55.515114,52.433029,164.453027,1471.898064
4,5,324.728038,91.253685,221.069806,1436.260392
...,...,...,...,...,...
13753,13754,206.419432,212.102989,10.116867,234.888696
13754,13755,252.961215,197.694189,313.463300,412.632696
13755,13756,265.635180,65.784569,393.811699,170.012136
13756,13757,193.609296,72.832144,206.992864,160.413960


In [6]:
df.to_csv(out_roi)

In [7]:
### Remove bad ROIs and save the file
# df_filtered = df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered = df.set_index('roi').copy()
df_filtered = df_filtered.loc[df_filtered.index.difference(bad_roi_list)]
df_filtered.to_csv(out_goodroi)

In [8]:
# ### Lipofuscin subtraction (optional)
# # Load spot files for lipofuscin subtraction
# channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
# channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# # Identify lipofuscin spots
# c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# # Remove lipofuscin spots
# true_pos_c0 = np.delete(channel_1, pAind, axis=0)
# true_pos_c1 = np.delete(channel_2, pBind, axis=0)

### spot extraction and save file
spotcount = spot.spot_counts(lb, spot_dir, verbose=False)
spotcount.to_csv(out_spots)

Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c0.txt
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c0.txt line# 79374
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 59548
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 293582
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 313383
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 320791
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 321804
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line# 323138
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\spots_pooled\spots_r1_c1.txt line#

In [9]:
### Intensity measurement and save file
df_intensity=intensity.measure_intensity(lb, img_dir, 'c0')
df_intensity.to_csv(out_intensity)

In [10]:
test = pd.read_csv(out_spots, index_col=0)
for col in test:
    print(np.max(test[col]))
test

930.0
1615.0
1115.0
66.0
115.0
278.0
106.0
32.0


,spots_r1_c0,spots_r1_c1,spots_r1_c2,spots_r1_c4,spots_r2_c0,spots_r2_c1,spots_r2_c2,spots_r2_c4
1,34.0,46.0,37.0,25.0,0.0,1.0,0.0,0.0
2,4.0,13.0,0.0,0.0,0.0,2.0,2.0,1.0
3,5.0,20.0,0.0,1.0,2.0,5.0,0.0,0.0
4,3.0,14.0,0.0,2.0,0.0,4.0,0.0,1.0
5,5.0,23.0,0.0,4.0,14.0,12.0,0.0,2.0
...,...,...,...,...,...,...,...,...
13754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13755,0.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0
13756,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0
13757,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
